### Evaluating Performance

**1. Weather Model**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

In [3]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
weather = pd.read_sql_query('select * from weatherinszeged',con=engine)
engine.dispose()

In [4]:
weather.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


Lets use the difference of 'temperature' and 'apparenttemperature' as the explained variable and 'humidity' and 'windspeed' as the explarotary variables and fit the OLS model and test the goodness of fit of the model.

In [7]:
X = weather[['humidity','windspeed']]
Y = weather['apparenttemperature'] - weather['temperature']  

In [8]:
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Sun, 15 Sep 2019   Prob (F-statistic):               0.00
Time:                        17:52:35   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

* The R-square and R-square adjusted values are equal and 0.288.This values is low which is close to zero than to 1.Therefore they are not satisfactory values.

* now lets add a feature with the interaction of the two independent variables in our previous model namely 'humidity' and 'windspeed' and run the OLS model again.

In [9]:
weather['humidity_windspeed'] = weather['humidity'] * weather['windspeed']

In [10]:
X = weather[['humidity','windspeed', 'humidity_windspeed']]
Y = weather['apparenttemperature'] - weather['temperature']  

In [11]:
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Sun, 15 Sep 2019   Prob (F-statistic):               0.00
Time:                        18:01:34   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0839      0.033      2.511      0.012       0.018       0.149
humidity               0.1775      0.043      4.133      0.000       0.093       0.262
windspeed              0.0905      0.002     36.797      0.000       0.086       0.095
humidity_windspeed    -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

We can see that our model performed better with an increase in the R-squared values(0.341).But still not sufficient though.

* Lets now add 'visibility' variable as a feature to our equation and run the first model again.

In [12]:
X = weather[['humidity','windspeed', 'visibility']]
Y = weather['apparenttemperature'] - weather['temperature']  

In [13]:
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                 1.401e+04
Date:                Sun, 15 Sep 2019   Prob (F-statistic):               0.00
Time:                        18:11:42   Log-Likelihood:            -1.6938e+05
No. Observations:               96453   AIC:                         3.388e+05
Df Residuals:                   96449   BIC:                         3.388e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5756      0.028     56.605      0.000       1.521       1.630
humidity      -2.6066      0.025   -102.784      0.000      -2.656      -2.557
windspeed     -0.1199      0.001   -179.014      0.000      -0.121      -0.119
visibility     0.0540      0.001     46.614      0.000       0.052       0.056
==============================================================================
Omnibus:                     3833.895   Durbin-Watson:                   0.282
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4584.022
Skew:                          -0.459   Prob(JB):                         0.00
Kurtosis:                       3.545   Cond. No.                         131.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The R-square is 0.304 and Adjusted R-squared is 0.303.Our model improved in performance as the R-values increased from the first model.But when compared with the interaction model,it performed less as the R-values turned out to be less.Therefore the interaction seems to be more useful.

* Comparing the above three model with respect to their AIC/BIC values,Model 2(the interaction model) has the lowest value of the AIC/BIC values which theoretically indicates its more useful model.This claim can be supported by the fact that, that model have the highest R-square and R-square adjusted value as compared to the other two models. 

**2.House prices model**

In [14]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(postgres_user, postgres_pw, postgres_host
                                                            ,postgres_port,postgres_db))
house_prices = pd.read_sql_query('select * from houseprices', con = engine)
engine.dispose()

In [15]:
house_prices.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [16]:
house_prices['is_z_centralair'] = pd.get_dummies(house_prices.centralair, drop_first=True)
num_columns = ['overallqual','totalbsmtsf', 'firstflrsf', 'grlivarea','garagecars', 'garagearea']
num_columns.append('is_z_centralair')
X = house_prices[num_columns]
Y = house_prices['saleprice']

In [20]:
house_prices['mszoning'].value_counts()

RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: mszoning, dtype: int64

In [17]:
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.763
Model:                            OLS   Adj. R-squared:                  0.762
Method:                 Least Squares   F-statistic:                     668.4
Date:                Sun, 15 Sep 2019   Prob (F-statistic):               0.00
Time:                        18:32:19   Log-Likelihood:                -17493.
No. Observations:                1460   AIC:                         3.500e+04
Df Residuals:                    1452   BIC:                         3.504e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const           -1.106e+05   5685.210    -19.456      0.000   -1.22e+05   -9.95e+04
overallqual      2.351e+04   1095.574     21.456      0.000    2.14e+04    2.57e+04
totalbsmtsf        23.5468      4.319      5.452      0.000      15.075      32.019
firstflrsf         11.5981      5.023      2.309      0.021       1.745      21.451
grlivarea          43.8685      2.687     16.326      0.000      38.598      49.139
garagecars       1.429e+04   3012.969      4.742      0.000    8377.231    2.02e+04
garagearea         14.5942     10.459      1.395      0.163      -5.922      35.110
is_z_centralair  1.192e+04   4334.626      2.749      0.006    3414.876    2.04e+04
==============================================================================
Omnibus:                      424.543   Durbin-Watson:                   1.981
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            39500.235
Skew:                          -0.253   Prob(JB):                         0.00
Kurtosis:                      28.477   Cond. No.                     1.46e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.46e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

* Running the house prices model again and assessing the goodness of fit of our model using F-test, R-squared, adjusted R-squared, AIC and BIC.
 * F-test: in this test the p-value is close to zero and this shows nothing but the features have a significant effect in explaining variation in housing price.
 * R-squared: 0.763,this value shows that our model is useful in predicting house pricing.
 * Adjusted r-squared: 0.762 has the same function as R-square in checking for goodness of fit of our model.
 * AIC/BIC: 3.500e+04/3.504e+04.At this stage there is nothing to compare,but after accessing the values of AIC/BIC on other models we can use this values to compare with other models values and select the model with the lowest of these values as the best fit.

* From the R-squared and adj-R-squared,the model seems satsfactory except that the coefficient significance of 'garagearea' failed the t-test.And removing that feature from the model would be better.However there is room for the model to do better,so lets try adding an interaction feature from the 'overallqual' and 'garagecars' and remove the 'garagearea' and run the model again.

In [18]:
house_prices['is_z_centralair'] = pd.get_dummies(house_prices.centralair, drop_first=True)
house_prices['averallqual_garagearea'] = house_prices['overallqual'] * house_prices['garagecars']
num_columns = ['overallqual','totalbsmtsf', 'firstflrsf', 'grlivarea','garagecars','averallqual_garagearea']
num_columns.append('is_z_centralair')
X = house_prices[num_columns]
Y = house_prices['saleprice']

In [19]:
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.805
Model:                            OLS   Adj. R-squared:                  0.804
Method:                 Least Squares   F-statistic:                     855.5
Date:                Mon, 16 Sep 2019   Prob (F-statistic):               0.00
Time:                        00:38:39   Log-Likelihood:                -17351.
No. Observations:                1460   AIC:                         3.472e+04
Df Residuals:                    1452   BIC:                         3.476e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   4.375e+04   1.02e+04      4.306      0.000    2.38e+04    6.37e+04
overallqual            -5534.4215   1917.606     -2.886      0.004   -9295.996   -1772.847
totalbsmtsf               16.8448      3.925      4.292      0.000       9.146      24.543
firstflrsf                10.1977      4.551      2.241      0.025       1.270      19.126
grlivarea                 42.7626      2.437     17.545      0.000      37.981      47.544
garagecars             -6.444e+04   4912.152    -13.118      0.000   -7.41e+04   -5.48e+04
averallqual_garagearea  1.455e+04    823.221     17.680      0.000    1.29e+04    1.62e+04
is_z_centralair         2.613e+04   4010.702      6.515      0.000    1.83e+04     3.4e+04
==============================================================================
Omnibus:                      513.388   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            40190.632
Skew:                          -0.716   Prob(JB):                         0.00
Kurtosis:                      28.664   Cond. No.                     2.75e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.75e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

The second model outperformed the first one.We can prove that with all the evaluation metrices.The R-squared value was raise from 0.763 to 0.805 and the AIC/BIC values were lower in the second model.Therefore the second model is the best from the two.

Lets keep trying to find more robust model by adding another categorical variable 'mszoning'.

In [26]:
new_house_prices = pd.concat([house_prices, pd.get_dummies(house_prices.mszoning, drop_first= True)], axis = 1)
dummy_col_names = list(pd.get_dummies(new_house_prices.mszoning, drop_first=True).columns)

so lets add those new columns in two our modeling data;

In [29]:
X = new_house_prices[num_columns + dummy_col_names]
Y = new_house_prices['saleprice']

In [30]:
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.811
Model:                            OLS   Adj. R-squared:                  0.810
Method:                 Least Squares   F-statistic:                     566.0
Date:                Mon, 16 Sep 2019   Prob (F-statistic):               0.00
Time:                        01:17:23   Log-Likelihood:                -17327.
No. Observations:                1460   AIC:                         3.468e+04
Df Residuals:                    1448   BIC:                         3.474e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   2.701e+04    1.4e+04      1.928      0.054    -463.971    5.45e+04
overallqual            -5957.3647   1907.281     -3.123      0.002   -9698.694   -2216.035
totalbsmtsf               17.0778      3.869      4.414      0.000       9.488      24.668
firstflrsf                 6.1279      4.561      1.343      0.179      -2.820      15.076
grlivarea                 43.5397      2.405     18.101      0.000      38.821      48.258
garagecars             -6.542e+04   4841.782    -13.512      0.000   -7.49e+04   -5.59e+04
averallqual_garagearea  1.459e+04    811.654     17.975      0.000     1.3e+04    1.62e+04
is_z_centralair         2.059e+04   4059.602      5.071      0.000    1.26e+04    2.86e+04
FV                      3.176e+04   1.22e+04      2.605      0.009    7843.096    5.57e+04
RH                      2.108e+04   1.41e+04      1.497      0.135   -6535.114    4.87e+04
RL                       3.22e+04   1.13e+04      2.846      0.004       1e+04    5.44e+04
RM                      1.422e+04   1.14e+04      1.247      0.213   -8149.676    3.66e+04
==============================================================================
Omnibus:                      486.492   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            42305.358
Skew:                          -0.582   Prob(JB):                         0.00
Kurtosis:                      29.345   Cond. No.                     6.36e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.36e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

So from the above result we might think at a glance that this model outperformed the two previous models.But when you looked more closer you find out that this might not be the case.From the t-test we can see that three of the coefficients of our model has insignificant effect.Not only that,we can also observe that the AIC is smaller than the previous model while the BIC is larger.This in consistency could be either a redflag for overfitting(AIC) or caused by the features parsimonious behaviour of the BIC.Lets re-run our model by removing the insignificant features to investigate more.

In [32]:
num_columns = ['overallqual','totalbsmtsf', 'grlivarea','garagecars','averallqual_garagearea','FV', 'RL']
num_columns.append('is_z_centralair')
X = new_house_prices[num_columns]
Y = new_house_prices['saleprice']

In [33]:
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.811
Model:                            OLS   Adj. R-squared:                  0.810
Method:                 Least Squares   F-statistic:                     777.0
Date:                Mon, 16 Sep 2019   Prob (F-statistic):               0.00
Time:                        15:46:54   Log-Likelihood:                -17329.
No. Observations:                1460   AIC:                         3.468e+04
Df Residuals:                    1451   BIC:                         3.472e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   4.185e+04   9919.261      4.219      0.000    2.24e+04    6.13e+04
overallqual            -5938.6859   1893.424     -3.136      0.002   -9652.827   -2224.545
totalbsmtsf               20.7966      2.630      7.908      0.000      15.638      25.955
grlivarea                 44.7953      2.235     20.043      0.000      40.411      49.179
garagecars             -6.521e+04   4841.900    -13.467      0.000   -7.47e+04   -5.57e+04
averallqual_garagearea  1.456e+04    810.871     17.952      0.000     1.3e+04    1.61e+04
FV                      1.712e+04   5051.688      3.388      0.001    7206.414     2.7e+04
RL                      1.852e+04   2607.748      7.100      0.000    1.34e+04    2.36e+04
is_z_centralair         2.071e+04   4017.547      5.155      0.000    1.28e+04    2.86e+04
==============================================================================
Omnibus:                      487.995   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            42467.338
Skew:                          -0.587   Prob(JB):                         0.00
Kurtosis:                      29.395   Cond. No.                     2.32e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.32e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

We can see that with decreased number of features the BIC little bit lowered from 4.74e+04 to 4.72e+04.But the AIC is still lower than the previous information criterion.Therefore this model is the best from the prvious three models.